Results for:

In [1]:
AGENT_NAME = 'Models/ATLA/PPO agent 20 alts over 250 1-9-21.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SAVE_DIR = 'PPO agent 20 alts over 250 1-9-21 results/'
ATK_NAME = 'untargeted_mybb'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from KBMproject.mybb import BrendelBethgeAttack as BBA

import pandas as pd
import numpy as np

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

Define RL agent

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [5]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [6]:
cols = env.observation_names

In [7]:
observation_masks = pd.read_csv('observation_masks.csv')

In [10]:
%%time
#%%capture
attack = utils.define_attack(agent, BBA)
kpi, obs, adv_obs, asr = utils.eval_untargeted_attack(agent, env, attack,mask=observation_masks['mask_time'].to_numpy())

CPU times: total: 1h 2min 34s
Wall time: 7h 16min 16s


In [11]:
asr

1.0

In [12]:
kpi

cost_function
annual_peak_average                      1.300127
carbon_emissions_total                   1.017678
cost_total                               0.965902
daily_one_minus_load_factor_average      1.000899
daily_peak_average                       1.115070
electricity_consumption_total            1.020469
monthly_one_minus_load_factor_average    1.013903
ramping_average                          1.511707
zero_net_energy                          1.069147
Name: District, dtype: float64

In [13]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename)
    df_kpis[ATK_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print('KPIs.csv updated')
except:
    kpi.name = ATK_NAME
    kpi.to_csv(kpi_savename)
    print('KPIs.csv created')

KPIs.csv updated


In [17]:
df_kpis = pd.read_csv(kpi_savename)
df_kpis[ATK_NAME] = kpi.values
df_kpis.to_csv(kpi_savename)

In [14]:
df_obs = pd.DataFrame(obs)
df_obs.columns = cols
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_obs.csv')

In [15]:
df_obs = pd.DataFrame(adv_obs)
df_obs.columns = cols
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_adv_obs.csv')

In [16]:
asr_savename = SAVE_DIR+'ASRs.csv'
try:
    df_asrs = pd.read_csv(asr_savename)
    df_asrs[ATK_NAME] = asr
    df_asrs.to_csv(asr_savename)
    print('ASRs.csv updated')
except:
    asr = pd.Series([asr])
    asr.name = ATK_NAME
    asr.to_csv(asr_savename)
    print('ASRs.csv created')

KPIs.csv created
